## 1.2 mongodb

* Why nosql - 모델링 schema 없이 (schema free) 문서를 저장.
    * Database - mongo는 데이터베이스를 생성하는 명령어가 없다. mongodb는 bson으로 저장
    * Collections (RDB table)
    * Document (RDB record)

RDBMS | MongoDB
-------|-------
Database | Database
Table | Collection
Row | Document
primary key | primary key (_id를 기본으로 제공)
schema | schema free

* install
    * linux
        ```
        sudo apt-get install mongodb-org
        sudo pip install pymongo
        ```
    * 윈도우
        * 다운로드 32,64비트 선택, msi 설치

* config 설정
    * 리눅스는 /etc/mongodb.conf
    * 윈도우
    * 설정은 YAML 또는 init 형식으로 한다.
    * 예:
        ```
        fork = true
        bind_ip = 127.0.0.1
        port = 27017
        quiet = true
        dbpath = /var/lib/mongodb
        logpath = /var/log/mongodb/mongod.log
        logappend = true
        journal = true
        ```

* db 저장 디렉토리 설정변경
    * 설정 파일 '/etc/mongodb.conf' 에서 변경할 수 있다. 기본 디렉토리는 '/data/db' 현재 설정은:
        ```
        dbpath=/var/lib/mongodb
        ```
        
    * cli로 변경
        ```
        mongod --dbpath
        ```

* 실행
    * linux
        * cli (config파일, db경로 설정해서)
            ```
            mongod --config ./mongodb.conf --dbpath ./data --port 27017


            ```
        * daemon
            ```
            sudo service mongodb start
            sudo service mongodb stop
            ```
 
    * 윈도우에서 실행은 경로설정을 한 후에 한다.
        * 제어판에서 하는 것이 편함
            ```
            setx mongodb "C:\Program Files\MongdDB\Server\3.2"
            setx path "%path%";"%mongodb%\bin"
            mongod
            ```
        * 32비트 설치 후 실행하는 경우, storageEngine오류 발생. 오류메시지 읽어보고 따라 함.
            ```
            mongod --storageEngine=mmapv1 --dbpath [your-path]
            ```
    * 문제가 있으면 위 명령문에 --repair 붙여서 한 후, --repair 빼고 다시 실행

### 1.2.1 mongo shell

* mongo cli
    ```
    mongo -u new_user -p new_password new_database
    ```

* mongofiles cli
    ```
    mongofiles list
    ```

* help는 --help

* sample cli
    ```
    $ mongo
    > show dbs
    > use test
    > db.createCollection("myCollection")
    > show collections
    > db.getCollectionNames() or show collections or show tables
    > db.dropDatabase()
    > db.help()
    ```


* cli 명령어


구분 | 예 | 설명
-------|-------
db create | > use DB_NAME | use mydb
db insert | > db.mydb.insert( {"name":"js"} ) | 데이터 입력
drop db | > use mydb <br> > db.dropDatabase() | mydb를 삭제함
create Collection | > db.createCOllection("mycollection") | mycollection을 생성
drop collection | db.mycollection.drop() | mycollection을 삭제
insert into collection | db.mycollection.insert( {"name":"js"} ) | 데이터 입력
read | db.mycollection.find() | 데이터 검색 limit(), sort()
read | db.mycol.find({"likes":{$lt:50}}) | 조건 검색 (less than)
update | db.mycol.update( {'name':'js 1'} ) | 갱신
remove | db.mycol.remove() | 삭제 1건
remove | db.mycol.remove( {'name':'js' } ) | 조건 삭제


* admin

예 | 설명
-------|-------
show dbs | show dbs show db list
show collections | show collection list


* mongod는 쉘 (mongod는 mongo daemon)

In [ ]:
!mongo

MongoDB shell version: 2.6.10
connecting to: test
> 

In [5]:
%%writefile src/ds_sql_mongo_test.js
db.test.save({x:1})
db.test.find()
db.stats()

Overwriting src/ds_sql_mongo_test.js


In [6]:
!mongo < src/ds_sql_mongo_test.js

MongoDB shell version: 2.6.10
connecting to: test
WriteResult({ "nInserted" : 1 })
{ "_id" : ObjectId("581121af5010767f01441c77"), "a" : 1 }
{ "_id" : ObjectId("581122e4e994a927d3601173"), "a" : 1 }
{ "_id" : ObjectId("581122f5136ebba424709b37"), "x" : 1 }
{ "_id" : ObjectId("5811232186d1fbf960da31b9"), "x" : 1 }
{
	"db" : "test",
	"collections" : 5,
	"objects" : 18,
	"avgObjSize" : 65.77777777777777,
	"dataSize" : 1184,
	"storageSize" : 40960,
	"numExtents" : 5,
	"indexes" : 5,
	"indexSize" : 40880,
	"fileSize" : 67108864,
	"nsSizeMB" : 16,
	"dataFileVersion" : {
		"major" : 4,
		"minor" : 5
	},
	"extentFreeList" : {
		"num" : 0,
		"totalSize" : 0
	},
	"ok" : 1
}
bye


## 문제 mongo shell

* javascript 파일을 저장하고 실행한다.

In [7]:
%%writefile src/ds_sql_mongo_cli.js
db
use Employees
show tables
db.mytable.find({ "id": 1 })

Writing src/ds_sql_mongo_cli.js


In [9]:
!mongo < src/ds_sql_mongo_cli.js

MongoDB shell version: 2.6.10
connecting to: test
test
switched to db Employees
mytable
system.indexes
{ "_id" : ObjectId("57be858bf5e6e917172543b8"), "country" : "ko", "age" : 11, "id" : 1, "name" : "js" }
bye


### 1.2.2. pymongo

* 설치
    * 리눅스 pip install pymongo
    * 윈도우
        ```
        conda install pymongo
        ```
* 실행
    * mongod가 백그라운드에서 실행되고 있어야 한다 (대기 상태 실행)
    * 아래 문제와 같이 pymongo 세션을 시작하면, 백그라운드 mongod가 "connection accpeted from localhost..."
    * connection은 close할 수 있다.

## 문제 mongo crud in python

* Employees 데이터베이스를 생성하고, 그 안에 mytable을 생성한다.


In [2]:
import pymongo

In [21]:
from pymongo import MongoClient
client = MongoClient('localhost:27017')
db=client.Employees

* insert

In [18]:
_id=1
_name='js'
_age=11
_country='ko'

In [22]:
db.mytable.insert({
    "id": _id,
    "name": _name,
    "age": _age,
    "country": _country
})

ObjectId('57be858bf5e6e917172543b8')

In [5]:
db.Employees.insert_one(
        {
        "id": _id,
        "name": _name,
        "age": _age,
        "country": _country
        })

* read

In [6]:
empCol = db.Employees.find()
for emp in empCol:
        print emp

{u'country': u'kor', u'age': 11, u'_id': ObjectId('57b6ed92f5e6e95aafa8bfcc'), u'id': 1, u'name': u'jsl'}


* update

In [7]:
_name='jslim'
_age=123
db.Employees.update_one(
        {"id": _id},
        {
        "$set": {
            "name":_name,
            "age":_age,
        }
        }
    )

In [8]:
empCol = db.Employees.find()
for emp in empCol:
        print emp

{u'country': u'kor', u'age': 123, u'_id': ObjectId('57b6ed92f5e6e95aafa8bfcc'), u'id': 1, u'name': u'jslim'}


* delete

In [ ]:
db.Employees.delete_many({"id":criteria})